# Dropout
---
Dropout prevents overfitting by randomly shutting down some output units. The video from Coursera vividly illustrate the process.

<center>
<video width="600" height="500" src="images/dropout1_kiank.mp4" controls>
</video>
</center>

In the process above, in each iteration, some units on layer `[2]` would be randomly muted, meaning that there would be less neurons working in the forward process, thus the overall structure of neural network is simplified. Meanwhile the trained model would be more robust, since the model no longer can rely on any specific neurons anymore (as they could be muted in the process), all other neurons would need to learn in the training. 

# Foward
---
You can think of dropout as adding an extra layer to the forward process.

In the previous sessions, we have the forward equations as following,

__Without Dropout__

$$ Z^{[l]} = W^{[l]}A^{[l-1]} + b^{[l]} $$

$$ A^{[l]} = g^{[l]}(Z^{[l]})$$

Where $g$ is the activation function. Now with dropout an extra layer is applied to $A^{[l]}$.

__Dropout__

$$ Z^{[l]} = W^{[l]}A^{[l-1]} + b^{[l]} $$

$$ A^{[l]} = g^{[l]}(Z^{[l]})$$

$$ A^{[l]} = D^{[l]}(A^{[l]})$$

Where $D$ is the dropout layer. The key factor in the dropout layer is `keep_prob` parameter, which specifies the probability of keeping each unit. Say if `keep_prob = 0.8`, we would have 80% chance of keeping each output unit as it is, and 20% chance set them to 0.

The implementation would be adding an extra mask to the result $A$. Assume we have an output $A^{[l]}$ with four elements as following,

$$ \begin{pmatrix}
a_1^{[l]} \\
a_2^{[l]} \\
a_3^{[l]} \\
a_4^{[l]}
\end{pmatrix}$$

And we want to mute the third unit while keeping the rest, what we need is a matrix of the same shape and do a element-wise multiplication as following,

$$ \begin{pmatrix}
a_1^{[l]} \\
a_2^{[l]} \\
a_3^{[l]} \\
a_4^{[l]}
\end{pmatrix} * 
\begin{pmatrix}
1 \\
1 \\
0 \\
1
\end{pmatrix} = 
\begin{pmatrix}
a_1^{[l]} \\
a_2^{[l]} \\
0 \\
a_4^{[l]}
\end{pmatrix}
$$

Let's first initialize some weight parameters.

In [1]:
import numpy as np

layers = [3, 10, 1]
def weights_init():
    params = {}
    n = len(layers)
    for i in range(1, n):
        params['W' + str(i)] = np.random.randn(layers[i], layers[i-1])*0.01
        params['b' + str(i)] = np.zeros((layers[i], 1))
    return params

In [2]:
params = weights_init()
params

{'W1': array([[ 0.01273373, -0.0029192 , -0.02510443],
        [-0.00323293,  0.01198572, -0.0127173 ],
        [ 0.00092662,  0.00766214,  0.02222858],
        [-0.01368964, -0.02118227,  0.01315665],
        [-0.02065367,  0.01095289,  0.00727299],
        [-0.00980028, -0.02437653, -0.0162406 ],
        [ 0.00637576, -0.02312436, -0.000291  ],
        [-0.0029315 ,  0.01407064,  0.00237895],
        [-0.00581215, -0.00695063,  0.00948468],
        [-0.00774545, -0.008947  ,  0.01390741]]),
 'b1': array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]),
 'W2': array([[ 0.00217584,  0.01116851, -0.01580682,  0.00626901, -0.0053493 ,
          0.01537351,  0.00633889,  0.0061288 ,  0.01380906, -0.00308319]]),
 'b2': array([[0.]])}

# Forward

In [3]:
keep_probs = [.5]

def sigmoid(x):
    return 1/(1 + np.exp(-x))


def relu(x):
    return np.maximum(x, 0)


def forward(X):
    # intermediate layer use relu as activation
    # last layer use sigmoid
    n_layers = int(len(params)/2)
    A = X
    cache = {}
    for i in range(1, n_layers):
        W, b = params['W'+str(i)], params['b'+str(i)]
        Z = np.dot(W, A) + b
        A = relu(Z)
        # dropout
        keep_prob = keep_probs[i-1]
        D = np.random.rand(A.shape[0], A.shape[1])
        D = (D < keep_prob).astype(int)
        A = np.multiply(D, A)
        # rescale
        A = A/keep_prob
        
        cache['Z'+str(i)] = Z
        cache['A'+str(i)] = A
        cache['D'+str(i)] = D

    # last layer
    W, b = params['W'+str(i+1)], params['b'+str(i+1)]
    Z = np.dot(W, A) + b
    A = sigmoid(Z)
    
    cache['Z'+str(i+1)] = Z
    cache['A'+str(i+1)] = A

    return cache, A

In [4]:
X = np.array([[1.2], [3], [-2]])
cache, _ = forward(X)

In [5]:
cache

{'Z1': array([[ 0.05673173],
        [ 0.05751222],
        [-0.02035879],
        [-0.10628767],
        [-0.00647173],
        [-0.05240875],
        [-0.06114016],
        [ 0.03393622],
        [-0.04679583],
        [-0.06395034]]),
 'A1': array([[0.        ],
        [0.11502445],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ]]),
 'D1': array([[0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0]]),
 'Z2': array([[0.00128465]]),
 'A2': array([[0.50032116]])}

Our layers is set to [3, 10, 1], where 3 is the input layer and 1 is the output layer. In the example above we give the hidden layer a `keep_prob` ratio of `0.5`, so some of the units are muted.

(__Note__: After dropout, `A` needs to rescale to `A = A / keep_prob`, since some of the units are disabled, the left units need to be amplified in order to match the expected value)

# Backward
---
The backward process is to mask the same function `D` to the corresponding `dA`.

In [6]:
# dummy code, full version needs to be inside a Class
def backward(self, cache, X, Y, keep_probs):
    """
    cache: result [A, Z]
    Y: shape (1, m)
    """
    grad = {}
    n_layers = int(len(self.params)/2)
    m = Y.shape[1]
    cache['A0'] = X

    for l in range(n_layers, 0, -1):
        A, A_prev, Z = cache['A' + str(l)], cache['A' + str(l-1)], cache['Z' + str(l)]
        W = self.params['W'+str(l)]
        if l == n_layers:
            dA = -np.divide(Y, A) + np.divide(1 - Y, 1 - A)

        if l == n_layers:
            dZ = np.multiply(dA, self.sigmoid_grad(A, Z))
        else:
            # dropout version
            D = cache['D' + str(l)]
            dA = np.multiply(dA, D)
            # rescale
            dA = dA/keep_probs[l-1]
            
            dZ = np.multiply(dA, self.relu_grad(A, Z))
        dW = np.dot(dZ, A_prev.T)/m
        db = np.sum(dZ, axis=1, keepdims=True)/m
        dA = np.dot(W.T, dZ)

        grad['dW'+str(l)] = dW
        grad['db'+str(l)] = db

    return grad

Note that in back propagation, $dA$ also needs to be rescaled

Now let's put everything in a class and apply it on a classification task.

In [7]:
import numpy as np


class deepNN:
    def __init__(self, layers):
        self.layers = layers
        self.params = {}
        self.dropout = []
        self.A = 0
        self.Y = 0
       
    
    def weights_init(self):
        n = len(self.layers)
        for i in range(1, n):
            self.params['W' + str(i)] = np.random.randn(self.layers[i], self.layers[i-1])*0.01
            self.params['b' + str(i)] = np.zeros((self.layers[i], 1))
    
    @staticmethod
    def sigmoid(x):
        return 1/(1 + np.exp(-x))

    @staticmethod
    def relu(x):
        return np.maximum(x, 0)
    
    @staticmethod
    def compute_cost(A, Y):
        """
        For binary classification, both A and Y would have shape (1, m), where m is the batch size
        """
        assert A.shape == Y.shape
        m = A.shape[1]
        s = np.dot(Y, np.log(A.T)) + np.dot(1-Y, np.log((1 - A).T))
        loss = -s/m
        return np.squeeze(loss)
    
    @staticmethod
    def sigmoid_grad(A, Z):
        grad = np.multiply(A, 1-A)
        return grad

    @staticmethod
    def relu_grad(A, Z):
        grad = np.zeros(Z.shape)
        grad[Z>0] = 1
        return grad
    
    
    def forward(self, X):
        # intermediate layer use relu as activation
        # last layer use sigmoid
        n_layers = int(len(self.params)/2)
        A = X
        cache = {}
        for i in range(1, n_layers):
            W, b = self.params['W'+str(i)], self.params['b'+str(i)]
            Z = np.dot(W, A) + b
            A = self.relu(Z)
            
            keep_prob = self.dropout[i-1]
            D = np.random.rand(A.shape[0], A.shape[1])
            D = np.int64(D < keep_prob)
            A = np.multiply(A, D)
            A = A/keep_prob
            
            cache['Z'+str(i)] = Z
            cache['A'+str(i)] = A
            cache['D'+str(i)] = D

        # last layer
        W, b = self.params['W'+str(i+1)], self.params['b'+str(i+1)]
        Z = np.dot(W, A) + b
        A = self.sigmoid(Z)
        cache['Z'+str(i+1)] = Z
        cache['A'+str(i+1)] = A

        return cache, A
    
    def backward(self, cache, X, Y):
        """
        cache: result [A, Z]
        Y: shape (1, m)
        """
        grad = {}
        n_layers = int(len(self.params)/2)
        m = Y.shape[1]
        cache['A0'] = X

        for l in range(n_layers, 0, -1):
            A, A_prev, Z = cache['A' + str(l)], cache['A' + str(l-1)], cache['Z' + str(l)]
            W = self.params['W'+str(l)]
            if l == n_layers:
                dA = -np.divide(Y, A) + np.divide(1 - Y, 1 - A)

            if l == n_layers:
                dZ = np.multiply(dA, self.sigmoid_grad(A, Z))
            else:
                keep_prob = self.dropout[l-1]
                D = cache['D' + str(l)]
                dA = np.multiply(dA, D)
                dA = dA/keep_prob
                dZ = np.multiply(dA, self.relu_grad(A, Z))
            dW = np.dot(dZ, A_prev.T)/m
            db = np.sum(dZ, axis=1, keepdims=True)/m
            dA = np.dot(W.T, dZ)

            grad['dW'+str(l)] = dW
            grad['db'+str(l)] = db

        return grad
    
    def optimize(self, grads, lr):
        n_layers = int(len(self.params)/2)
        for i in range(1, n_layers+1):
            dW, db = grads['dW'+str(i)], grads['db'+str(i)]
            self.params['W'+str(i)] -= lr*dW
            self.params['b'+str(i)] -= lr*db
    
    @staticmethod
    def generate_batch(X, batch_size):
        n = X.shape[0]
        batches = [range(i, i+batch_size) for i in range(0, n, batch_size)]
        return batches
    
    def train(self, X_train, y_train, batch_size=200, n_iter=100, lr=0.1, dropout:list=[]):
        self.dropout = dropout
        # prepare batch training
        batches = self.generate_batch(X_train, batch_size)
        # init weights
        self.weights_init()
        for i in range(n_iter):
            for batch in batches:
                X = X_train[batch, :].T
                Y = y_train[batch].reshape(1, -1)
                cache, A = self.forward(X)
                grads = self.backward(cache, X, Y)
                self.optimize(grads, lr)

            if i%10 == 0:
                loss = self.compute_cost(A, Y)
                print(f'iteration {i}: loss {loss}')


def accuracy(Y, Y_pred):
    """
    Y: vector of true value
    Y_pred: vector of predicted value
    """
    
    assert Y.shape[0] == 1
    assert Y.shape == Y_pred.shape
    Y_pred = np.round(Y_pred)
    acc = float(np.dot(Y, Y_pred.T) + np.dot(1 - Y, 1 - Y_pred.T))/Y.size
    return acc

In [8]:
from sklearn import datasets


X, y = datasets.make_classification(n_samples=10000, n_features=200, random_state=123)

X_train, X_test = X[:8000], X[8000:]
y_train, y_test = y[:8000], y[8000:]

print('train shape', X_train.shape)
print('test shape', X_test.shape)

train shape (8000, 200)
test shape (2000, 200)


In [9]:
layers = [200, 100, 20, 1]
dropout_ratio = [.8, .8]

model = deepNN(layers)

In [10]:
model.train(X_train, y_train, batch_size=200, n_iter=150, lr=0.02, dropout=dropout_ratio)

iteration 0: loss 0.6931117424217708
iteration 10: loss 0.6929932803338776
iteration 20: loss 0.6930042095652803
iteration 30: loss 0.6929270740459051
iteration 40: loss 0.6928159708817516
iteration 50: loss 0.6925835619958787
iteration 60: loss 0.6922472029699815
iteration 70: loss 0.690539000831825
iteration 80: loss 0.6813485269598472
iteration 90: loss 0.5140425561651449
iteration 100: loss 0.26596827522989325
iteration 110: loss 0.21297544130219212
iteration 120: loss 0.15886453417841173
iteration 130: loss 0.14315587310443184
iteration 140: loss 0.08922784883210816


In [11]:
_, pred = model.forward(X_test.T)
acc = accuracy(y_test.reshape(1, -1), pred)

print(f'accuracy {acc}')

accuracy 0.936
